In [174]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [175]:
prices = pd.read_csv('prices.csv')
wind = pd.read_csv('wind_power_cleaned.csv')
grid_excess = pd.read_csv('grid_excess.csv')

In [176]:
prices['date'] = pd.to_datetime(prices['date'])
wind['date'] = pd.to_datetime(wind['date'])

In [177]:
data = pd.merge(prices, wind, on=['date', 'hour'], how='outer')

#select every 18th day and save that to a new dataframe
data['date'] = pd.to_datetime(data['date'])

In [178]:
#show rows with nan values
data[data.isnull().any(axis=1)]

,date,hour,price,wind power
7160,2023-10-26,9,70.47,NaN
7161,2023-10-26,10,70.50,NaN
7162,2023-10-26,11,74.21,NaN
7163,2023-10-26,12,74.17,NaN
7223,2023-10-29,0,14.05,NaN
8760,2023-03-26,2,NaN,1293.8


In [179]:
#get a list of every 18th day in 2023
selected_dates = pd.date_range(start='2023-01-01', end='2023-12-31', freq='19D')

In [180]:
#select every 18th day
scenarios_price = {}
scenarios_wind = {}
i=1
for date in selected_dates:
    scenarios_price[i] = prices[prices['date'] == date].reset_index(drop=True).drop(columns=['date', 'hour'])
    scenarios_wind[i] = wind[wind['date'] == date].reset_index(drop=True).drop(columns=['date', 'hour'])
    i+=1

In [181]:
#make 20 times 20 times 2 scenarios
scenarios = {}
for price in scenarios_price:
    for wind in scenarios_wind:
        for grid in grid_excess:
            df = pd.merge(scenarios_price[price], scenarios_wind[wind], left_index=True, right_index=True)
            #add column with grid excess
            df['grid_excess'] = grid_excess[grid]
            scenarios[(price, wind, int(grid))] = df

In [182]:
import pickle 

with open('scenarios.pkl', 'wb') as f:
    pickle.dump(scenarios, f)
        
with open('scenarios.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)